In [60]:
import glob

# Importing drive method from colab for accessing google drive
from google.colab import drive

In [61]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import glob
import numpy as np
import pandas as pd 

In [63]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [64]:
import copy 

In [65]:
class Model() :
    network = [[1 for i in range(4)] for j in range(100000)]
    it =  0 

    def input(self,input) :
        self.input = input 
        
    def output(self,output) :
        self.output = output 
        
    def add(self,input_shape , prev_shape , activation_func) :
            weight = [[random() for i in range(prev_shape)] for j in range(input_shape)]
            neuron = [random() for i in range(input_shape)]
            bias = random()

            self.network[self.it][0],self.network[self.it][1],self.network[self.it][2],self.network[self.it][3] =  weight,neuron,bias,activation_func
            self.it+=1 
            
    
            
    
    

In [66]:
def conv2d(image,kernel) :
    
    #kernel.size = (3,3) ; 
    length = len(image)
    #print(length)
    output = [[[0,0,0]  for j in range(length-3)] for i in range(length-3) ]
    for i in range(0,length-3,1) :
        for j in range(0,length-3,1) :
            for k in range(3):
                ans = 0 
                for a in range(3):
                    for b in range(3) :
                        ans += image[i+a][j+b][k]*kernel[a][b]
                #print(ans)
                output[i][j][k] =  ans 
                
                
    return output

# (image[3:6][0:3][0]*kernel).sum()
     

In [67]:
filter = [[[1/9,1/9,1/9],[1/9,1/9,1/9],[1/9,1/9,1/9]],[[0,-1,0],
                                                       [-1,5,-1],[0,-1,0]],[[1 ,0,1],[-2,0,2],[-1,0,1]],[[1,-2,-1],[0,0,0],[1,2,1]]]

In [68]:
def pooling(feature_map, size=2, stride=2):    
    pool_out = np.zeros((np.uint16((feature_map.shape[0]-size+1)/stride),  
                              np.uint16((feature_map.shape[1]-size+1)/stride),  
                              feature_map.shape[-1]))  
    for map_num in range(feature_map.shape[-1]):  
        r2 = 0  
        for r in np.arange(0,feature_map.shape[0]-size-1, stride):  
            c2 = 0  
            for c in np.arange(0, feature_map.shape[1]-size-1, stride):  
                pool_out[r2, c2, map_num] = np.max(feature_map[r:r+size,  c:c+size])  
                c2 = c2 + 1  
            r2 = r2 +1 
            
    return pool_out
                

FORWARD PROPAGATION

```
# This is formatted as code
```



In [69]:
def active_sum(input,weight,bias) :
    activation = bias 
    for i in range(len(weight)) : 
        activation += (weight[i]*input[i]) 
        
    return activation

In [70]:
def sigmoid(input) :
    return 1.0/(1.0 + np.exp(-input))

In [71]:
def ReLU(input) :
    if(input >0) :
        return input 
    return (0.01*input)

In [72]:
def Softmax(input) :
  output = [0 for i in range(len(input))]
  sum =  0 
  for i in range(len(input)) :
    sum += np.exp(input[i]) 
  for i in range(len(input)) :
    output[i] = (np.exp(input[i]))/sum 

  return output

In [73]:
def Activation(sum_active,function) :
    if(function == 'ReLU'):
        return ReLU(sum_active)
    if(function == 'sigmoid') :
        return sigmoid(sum_active)
    if(function == 'softmax') :
      return Softmax(sum_active)

In [74]:
def forward_prop(network,input,bias,it) :
    #print(it)
    my_input = input
    my_bias = bias 
    #print(my_input)
    for i in range(it-1) :  #Each hidden layer
        sum_active = [0 for i in range(len(network[i][1]))]
        sum_all = 0
        for j in range(len(network[i][1])): #For each neuron 
            sum_active[j] = active_sum(network[i][0][j],my_input,my_bias)
            sum_all += sum_active[j]
        
        sum_active = normalise(sum_active)
        ## Instead of normalise , I'll divide it by sum of all output 

        for j in range(len(network[i][1])) :
          #sum_active[j] = sum_active[j]/sum_all
          activation = Activation(sum_active[j],network[i][3])      
          network[i][1][j] = activation
        
        my_input = network[i][1]
        my_bias = network[i][2]         
            
    #### For last layer 
    sum_active1 = [0 for i in range(len(network[it-1][1]))]
    for j in range(len(network[it-1][1])) :
        sum_active1[j] = active_sum(network[it-1][0][j],my_input,my_bias)
    
    sum_active = normalise(sum_active)
    output = Softmax(sum_active1)
    network[it-1][1] = output 
        
            
    #output = Softmax(sum_active)
    #for j in range(len(network[it-1][1])): #For each neuron 
     #   network[it-1][1][j] = output[j]
                     
    return network

In [75]:
def sig_derivative(input) :
    
    return sigmoid(input)*(1.0  - sigmoid(input))

In [76]:
def rel_derivative(input) :
    if(input >0) :
        return 1 
    return (-0.01)

In [77]:
def soft_derivative(x) : # Complete layer is passed 
    y = [0 for i in range(len(x))] 
    for i in range(len(x)) :
        for j in range(len(x)) :
            if(i != j) :
                y[i] += x[i]*x[j]*(-1) 
            else :
                y[i] += x[i]*(1-x[i]) 
                
    return y

In [78]:
def derivative(input,function) :

    if(function == 'ReLU') :
       ## print("ReLu")
        return rel_derivative(input)
    if(function == 'sigmoid') :
        #print("Sigmoid")
        return sig_derivative(input)

      
        

In [79]:
def back_prop(network,input,output,learning_rate,bias,it) :
    error = [[1 for i in range(1000)] for j in range(it)]
    #For last layer 
    it-=1 
    sum_det = 0 #For bias 

    ### Calculation error and derivative for Softmax(last layer)
    error[it] =  soft_derivative(network[it][1]) 
    
    for j in range(len(error[it])) :
        sum_det += error[it][j] 

    for i in range(len(network[it][1])) : #For each neuron  
        #Now adjusting all weights to this neuron 
        delta = error[it][i]
        for j in range(len(network[it][0][i])) :
            network[it][0][i][j] += (learning_rate*delta*network[it-1][1][j])
            
            
    network[it-1][2] *= (1 + learning_rate*sum_det)
    
    it-=1 
    
    
    ## Now doing for all layers but first 
    while(it != 0) :
        sum_det = 0
        for i in range(len(network[it][1])) : #For each neuron 
            
            err = 0 #Calculating err for each neuron which is summation(error[it+1][j]*w[j][i])
            
            for j in range(len(network[it+1][1])) :
                err += (error[it+1][j]*network[it+1][0][j][i])
                
            
            error[it][i] = err 
            delta = error[it][i]*derivative(network[it][1][i],network[it][3]) 
            error[it][i] = delta
            sum_det += delta
            for j in range(len(network[it][0][i])) :
                network[it][0][i][j] += (learning_rate*delta*network[it-1][1][j])
            
            
        network[it-1][2] *= (1 + learning_rate*sum_det)

        it-=1 
        
        
    #Now for last layer . Here the only change will be that instead of network we'll take input and input bias 
    
    sum_det = 0
    for i in range(len(network[it][1])) : #For each neuron 

        err = 0 #Calculating err for each neuron which is summation(error[it+1][j]*w[j][i])

        for j in range(len(network[it+1][1])) :
            err += (error[it+1][j]*network[it+1][0][j][i])


        error[it][i] = err 
        delta = error[it][i]*derivative(network[it][1][i],network[it][3])  
        error[it][i] = delta
        sum_det += delta
        for j in range(len(network[it][0][i])) :
            network[it][0][i][j] += (learning_rate*delta*input[j])


    bias*= (1 + learning_rate*sum_det)
    
            
            
    
    return network
        
    

In [80]:
def normalise(input) :
    mean = sum(input) / len(input) 
    variance = sum([((x - mean) ** 2) for x in input]) / len(input)
    for k in range(len(input)) :
        input[k] = (input[k] - mean)/variance
        
    return input

In [81]:
def train(network,output,bias,learning_rate,it,epoch,num_img) :
    error = []
    
    for i in range(epoch):
        for j in range(num_img) :
            start_time = datetime.now() 
            answer = 0 
            a[j] = normalise(a[j]) 
            network = forward_prop(network,a[j],bias,it)
            network = back_prop(network,a[j],output[0],learning_rate,bias,it)
            
            #answer += error_function(network[it-1][1],output[0])
            #print("Time taken to run a is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            b[j] = normalise(b[j]) 
            network = forward_prop(network,b[j],bias,it)
            network = back_prop(network,b[j],output[1],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[1])
            
            #print("Time taken to run b is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            c[j] = normalise(c[j]) 
            network = forward_prop(network,c[j],bias,it)
            network = back_prop(network,c[j],output[2],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[2])
            #print("Time taken to run c is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            d[j] = normalise(d[j] ) 
            network = forward_prop(network,d[j] ,bias,it)
            network = back_prop(network,d[j] ,output[3],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[3])
            #print("Time taken to run d is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            e[j] = normalise(e[j]) 
            network = forward_prop(network,e[j],bias,it)
            network = back_prop(network,e[j],output[4],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[4])
            #print("Time taken to run e is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            f[j] = normalise(f[j]) 
            network = forward_prop(network,f[j],bias,it)
            network = back_prop(network,f[j],output[5],learning_rate,bias,it)
            #answer += error_function(network[it-1][1],output[5])
            #print("Time taken to run e is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            
            print("Image number ",j," done")
            print("Time taken to run Image ",j," is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            print(network[2][1])
                
        error.append(answer/epoch*num_img)
        
    
    return network,error

In [82]:

    
def predict(tmp,input,bias,it) :
  network = copy.deepcopy(tmp) 
  #print(it)
  my_input = input
  my_bias = bias 
  #print(my_input)
  for i in range(it-1) :  #Each hidden layer
      sum_active = [0 for i in range(len(network[i][1]))]
      sum_all = 0
      for j in range(len(network[i][1])): #For each neuron 
          sum_active[j] = active_sum(network[i][0][j],my_input,my_bias)
          sum_all += sum_active[j]
      
      sum_active = normalise(sum_active)
      ## Instead of normalise , I'll divide it by sum of all output 

      for j in range(len(network[i][1])) :
        #sum_active[j] = sum_active[j]/sum_all
        activation = Activation(sum_active[j],network[i][3])      
        network[i][1][j] = activation
      
      my_input = network[i][1]
      my_bias = network[i][2]         
          
  #### For last layer 
  sum_active1 = [0 for i in range(len(network[it-1][1]))]
  for j in range(len(network[it-1][1])) :
      sum_active1[j] = active_sum(network[it-1][0][j],my_input,my_bias)
  
  sum_active = normalise(sum_active)
  output = Softmax(sum_active1)
  network[it-1][1] = output 
        
            
    #output = Softmax(sum_active)
    #for j in range(len(network[it-1][1])): #For each neuron 
     #   network[it-1][1][j] = output[j]
                     
  return network[it-1][1]

In [83]:
def dataset_extraction(path,class_number):
    x_train=[]
    y_train=[]
    all_images=glob.glob(path+'/*.jpg')
    for path in all_images :
            img=load_img(path, target_size=(150,150))
            img=img_to_array(img)
            img = img/255
            x_train.append(img)
            y_train.append(class_number)
    return np.array(x_train),np.array(y_train)

In [84]:
from datetime import datetime 

In [85]:
start_time = datetime.now() 
trainX_building, trainY_building  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/buildings/",0)
trainX_forest,trainY_forest  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/forest/",1)
trainX_glacier,trainY_glacier  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/glacier/",2)
trainX_mount,trainY_mount  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/mountain/",3)
trainX_sea,trainY_sea  = dataset_extraction("/content/drive/My Drive/seg_test/seg_test/sea/",4)
trainX_street,trainY_street  =  dataset_extraction("/content/drive/My Drive/seg_test/seg_test/street/",5)

print('Time taken to run this cell is  (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Time taken to run this cell is  (hh:mm:ss.ms) 0:00:06.792926


In [86]:
import matplotlib.pyplot as plt 
%matplotlib inline

import math 
from random import random 

from PIL import Image 

import copy 

In [87]:
trainX_building.shape , trainX_forest.shape , trainX_glacier.shape , trainX_mount.shape, trainX_sea.shape , trainX_street.shape   

((437, 150, 150, 3),
 (474, 150, 150, 3),
 (553, 150, 150, 3),
 (525, 150, 150, 3),
 (510, 150, 150, 3),
 (501, 150, 150, 3))

In [88]:
a = [0 for i in range(400)]
b = [0 for i in range(400)]
c = [0 for i in range(400)]
d = [0 for i in range(400)]
e = [0 for i in range(400)]
f = [0 for i in range(400)]

In [89]:
trainX_forest = trainX_forest[:400]
trainX_glacier = trainX_glacier[:400]
trainX_mount = trainX_mount[:400]
trainX_sea = trainX_sea[:400]
trainX_street = trainX_street[:400]
trainX_building = trainX_building[:400]

In [90]:
def conv_preprocess(input) :
  a = conv2d(input,filter[2]) 
  #print(len(a))
  b = conv2d(a,filter[3]) 
  #print(len(b))
  c = pooling(np.array(b)) 
  #print(c.shape)
  d = conv2d(c,filter[1])
  #print(len(d))
  e = conv2d(d,filter[0])
  #print(len(e))
  f = pooling(np.array(e)) 
  #print(f.shape)
  f = np.array(f)
  return f.flatten().reshape(f.shape[0]*f.shape[1]*f.shape[2])

In [91]:
for i in range(60) :
  start_time = datetime.now() 
  a[i] = conv_preprocess(trainX_building[i])
  b[i] = conv_preprocess(trainX_forest[i])
  c[i] = conv_preprocess(trainX_glacier[i])
  d[i] = conv_preprocess(trainX_mount[i])
  e[i] = conv_preprocess(trainX_sea[i])
  f[i] = conv_preprocess(trainX_street[i])
  print('Time taken to run',i,"th it is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
  


Time taken to run 0 th it is cell is  (hh:mm:ss.ms) 0:00:17.522326
Time taken to run 1 th it is cell is  (hh:mm:ss.ms) 0:00:17.288946
Time taken to run 2 th it is cell is  (hh:mm:ss.ms) 0:00:17.285798
Time taken to run 3 th it is cell is  (hh:mm:ss.ms) 0:00:17.388451
Time taken to run 4 th it is cell is  (hh:mm:ss.ms) 0:00:17.144666
Time taken to run 5 th it is cell is  (hh:mm:ss.ms) 0:00:17.156473
Time taken to run 6 th it is cell is  (hh:mm:ss.ms) 0:00:17.416469
Time taken to run 7 th it is cell is  (hh:mm:ss.ms) 0:00:17.290905
Time taken to run 8 th it is cell is  (hh:mm:ss.ms) 0:00:17.166975
Time taken to run 9 th it is cell is  (hh:mm:ss.ms) 0:00:17.016257
Time taken to run 10 th it is cell is  (hh:mm:ss.ms) 0:00:17.172613
Time taken to run 11 th it is cell is  (hh:mm:ss.ms) 0:00:17.194268
Time taken to run 12 th it is cell is  (hh:mm:ss.ms) 0:00:17.624867
Time taken to run 13 th it is cell is  (hh:mm:ss.ms) 0:00:17.358125
Time taken to run 14 th it is cell is  (hh:mm:ss.ms) 0:00:

In [92]:
a[0].shape 

(3072,)

In [93]:
output= [[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]

In [112]:
model = Model()

In [113]:
model.add(512,3072,'ReLU')
model.add(70,512,'ReLU')
model.add(6,70,'softmax')

In [114]:
bias = 1
epoch = 1
learning_rate = 0.1
print(model.it)

3


In [115]:
model.network[2][1]

[0.15672002541408303,
 0.9710764317229573,
 0.9606846433794751,
 0.7610539667258922,
 0.6955036650433859,
 0.0824077511087048]

In [116]:
model.network , error = train(model.network,output,bias,learning_rate,model.it,epoch,50)

Image number  0  done
Time taken to run Image  0  is  (hh:mm:ss.ms) 0:00:08.917988
[1.8311155579914248e-17, 0.000391388269174158, 0.9994976838074741, 0.00011069912441054445, 1.1193579201016675e-09, 2.2767958334570864e-07]
Image number  1  done
Time taken to run Image  1  is  (hh:mm:ss.ms) 0:00:08.813674
[7.412357897262175e-21, 3.592136231692275e-09, 0.999964868755059, 3.3480852359469645e-05, 6.826718835355467e-14, 1.6468003771781116e-06]
Image number  2  done
Time taken to run Image  2  is  (hh:mm:ss.ms) 0:00:08.834614
[1.8319654419320438e-14, 6.427294292892214e-10, 0.9999836675920025, 1.6326868168135813e-05, 2.0384461006979297e-13, 4.896877830735205e-09]
Image number  3  done
Time taken to run Image  3  is  (hh:mm:ss.ms) 0:00:08.845689
[1.1233285092161568e-05, 0.0011318838981297786, 0.9818561652416964, 0.016053716097233675, 0.00017101005190852157, 0.0007759914259395438]
Image number  4  done
Time taken to run Image  4  is  (hh:mm:ss.ms) 0:00:13.218095
[3.619643278612383e-06, 0.0005038

In [124]:
def test_acc(list,network,bias,it):
  output = [0,0,0,0,0,0] 
  for i in range(20) :
    start_time = datetime.now() 
    ans = predict(network,list[i],bias,it)
    max = 0 
    max_ind = -1 
    for j in range(6) :
      if(ans[j] > max) :
        max = ans[j] 
        max_ind = j 

    output[max_ind]+=1 
    print('Time taken to run',i,"th it is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))

  return output 

In [125]:
OUT_A = test_acc(a,model.network,bias,model.it)

Time taken to run 0 th it is cell is  (hh:mm:ss.ms) 0:00:05.121190
Time taken to run 1 th it is cell is  (hh:mm:ss.ms) 0:00:04.931980
Time taken to run 2 th it is cell is  (hh:mm:ss.ms) 0:00:04.969014
Time taken to run 3 th it is cell is  (hh:mm:ss.ms) 0:00:04.934047
Time taken to run 4 th it is cell is  (hh:mm:ss.ms) 0:00:05.040240
Time taken to run 5 th it is cell is  (hh:mm:ss.ms) 0:00:04.987650
Time taken to run 6 th it is cell is  (hh:mm:ss.ms) 0:00:04.991987
Time taken to run 7 th it is cell is  (hh:mm:ss.ms) 0:00:04.941968
Time taken to run 8 th it is cell is  (hh:mm:ss.ms) 0:00:04.979262
Time taken to run 9 th it is cell is  (hh:mm:ss.ms) 0:00:04.901932
Time taken to run 10 th it is cell is  (hh:mm:ss.ms) 0:00:04.780412
Time taken to run 11 th it is cell is  (hh:mm:ss.ms) 0:00:04.993221
Time taken to run 12 th it is cell is  (hh:mm:ss.ms) 0:00:04.901284
Time taken to run 13 th it is cell is  (hh:mm:ss.ms) 0:00:05.087069
Time taken to run 14 th it is cell is  (hh:mm:ss.ms) 0:00:

In [126]:
OUT_A

[0, 0, 13, 7, 0, 0]

In [127]:
OUT_B = test_acc(b,model.network,bias,model.it)

Time taken to run 0 th it is cell is  (hh:mm:ss.ms) 0:00:04.962470
Time taken to run 1 th it is cell is  (hh:mm:ss.ms) 0:00:04.972620
Time taken to run 2 th it is cell is  (hh:mm:ss.ms) 0:00:04.748302
Time taken to run 3 th it is cell is  (hh:mm:ss.ms) 0:00:04.933569
Time taken to run 4 th it is cell is  (hh:mm:ss.ms) 0:00:04.851128
Time taken to run 5 th it is cell is  (hh:mm:ss.ms) 0:00:05.009377
Time taken to run 6 th it is cell is  (hh:mm:ss.ms) 0:00:04.992210
Time taken to run 7 th it is cell is  (hh:mm:ss.ms) 0:00:04.927901
Time taken to run 8 th it is cell is  (hh:mm:ss.ms) 0:00:04.944490
Time taken to run 9 th it is cell is  (hh:mm:ss.ms) 0:00:04.926034
Time taken to run 10 th it is cell is  (hh:mm:ss.ms) 0:00:04.874059
Time taken to run 11 th it is cell is  (hh:mm:ss.ms) 0:00:04.721220
Time taken to run 12 th it is cell is  (hh:mm:ss.ms) 0:00:04.926759
Time taken to run 13 th it is cell is  (hh:mm:ss.ms) 0:00:04.948204
Time taken to run 14 th it is cell is  (hh:mm:ss.ms) 0:00:

In [128]:
OUT_C = test_acc(c,model.network,bias,model.it)

Time taken to run 0 th it is cell is  (hh:mm:ss.ms) 0:00:04.988882
Time taken to run 1 th it is cell is  (hh:mm:ss.ms) 0:00:04.949644
Time taken to run 2 th it is cell is  (hh:mm:ss.ms) 0:00:04.981273
Time taken to run 3 th it is cell is  (hh:mm:ss.ms) 0:00:04.697048
Time taken to run 4 th it is cell is  (hh:mm:ss.ms) 0:00:04.926820
Time taken to run 5 th it is cell is  (hh:mm:ss.ms) 0:00:04.912323
Time taken to run 6 th it is cell is  (hh:mm:ss.ms) 0:00:04.926705
Time taken to run 7 th it is cell is  (hh:mm:ss.ms) 0:00:04.892111
Time taken to run 8 th it is cell is  (hh:mm:ss.ms) 0:00:04.928360
Time taken to run 9 th it is cell is  (hh:mm:ss.ms) 0:00:04.828006
Time taken to run 10 th it is cell is  (hh:mm:ss.ms) 0:00:04.884306
Time taken to run 11 th it is cell is  (hh:mm:ss.ms) 0:00:04.913995
Time taken to run 12 th it is cell is  (hh:mm:ss.ms) 0:00:04.998892
Time taken to run 13 th it is cell is  (hh:mm:ss.ms) 0:00:04.918875
Time taken to run 14 th it is cell is  (hh:mm:ss.ms) 0:00:

In [129]:
OUT_D = test_acc(d,model.network,bias,model.it)

Time taken to run 0 th it is cell is  (hh:mm:ss.ms) 0:00:04.916111
Time taken to run 1 th it is cell is  (hh:mm:ss.ms) 0:00:04.884614
Time taken to run 2 th it is cell is  (hh:mm:ss.ms) 0:00:04.903179
Time taken to run 3 th it is cell is  (hh:mm:ss.ms) 0:00:04.903222
Time taken to run 4 th it is cell is  (hh:mm:ss.ms) 0:00:04.950629
Time taken to run 5 th it is cell is  (hh:mm:ss.ms) 0:00:04.931814
Time taken to run 6 th it is cell is  (hh:mm:ss.ms) 0:00:05.081465
Time taken to run 7 th it is cell is  (hh:mm:ss.ms) 0:00:05.047935
Time taken to run 8 th it is cell is  (hh:mm:ss.ms) 0:00:04.750874
Time taken to run 9 th it is cell is  (hh:mm:ss.ms) 0:00:04.921081
Time taken to run 10 th it is cell is  (hh:mm:ss.ms) 0:00:04.949770
Time taken to run 11 th it is cell is  (hh:mm:ss.ms) 0:00:04.949338
Time taken to run 12 th it is cell is  (hh:mm:ss.ms) 0:00:04.903993
Time taken to run 13 th it is cell is  (hh:mm:ss.ms) 0:00:04.919332
Time taken to run 14 th it is cell is  (hh:mm:ss.ms) 0:00:

In [130]:
OUT_E = test_acc(e,model.network,bias,model.it)

Time taken to run 0 th it is cell is  (hh:mm:ss.ms) 0:00:04.926759
Time taken to run 1 th it is cell is  (hh:mm:ss.ms) 0:00:04.706388
Time taken to run 2 th it is cell is  (hh:mm:ss.ms) 0:00:05.002232
Time taken to run 3 th it is cell is  (hh:mm:ss.ms) 0:00:04.908803
Time taken to run 4 th it is cell is  (hh:mm:ss.ms) 0:00:04.967283
Time taken to run 5 th it is cell is  (hh:mm:ss.ms) 0:00:04.881729
Time taken to run 6 th it is cell is  (hh:mm:ss.ms) 0:00:05.043970
Time taken to run 7 th it is cell is  (hh:mm:ss.ms) 0:00:04.999440
Time taken to run 8 th it is cell is  (hh:mm:ss.ms) 0:00:04.999446
Time taken to run 9 th it is cell is  (hh:mm:ss.ms) 0:00:04.955149
Time taken to run 10 th it is cell is  (hh:mm:ss.ms) 0:00:05.020254
Time taken to run 11 th it is cell is  (hh:mm:ss.ms) 0:00:04.987357
Time taken to run 12 th it is cell is  (hh:mm:ss.ms) 0:00:04.958868
Time taken to run 13 th it is cell is  (hh:mm:ss.ms) 0:00:04.749599
Time taken to run 14 th it is cell is  (hh:mm:ss.ms) 0:00:

In [131]:
OUT_F = test_acc(f,model.network,bias,model.it)

Time taken to run 0 th it is cell is  (hh:mm:ss.ms) 0:00:04.983867
Time taken to run 1 th it is cell is  (hh:mm:ss.ms) 0:00:04.923034
Time taken to run 2 th it is cell is  (hh:mm:ss.ms) 0:00:04.906402
Time taken to run 3 th it is cell is  (hh:mm:ss.ms) 0:00:04.954932
Time taken to run 4 th it is cell is  (hh:mm:ss.ms) 0:00:04.945536
Time taken to run 5 th it is cell is  (hh:mm:ss.ms) 0:00:04.908704
Time taken to run 6 th it is cell is  (hh:mm:ss.ms) 0:00:04.697621
Time taken to run 7 th it is cell is  (hh:mm:ss.ms) 0:00:04.943717
Time taken to run 8 th it is cell is  (hh:mm:ss.ms) 0:00:04.945769
Time taken to run 9 th it is cell is  (hh:mm:ss.ms) 0:00:04.908300
Time taken to run 10 th it is cell is  (hh:mm:ss.ms) 0:00:04.929312
Time taken to run 11 th it is cell is  (hh:mm:ss.ms) 0:00:04.925730
Time taken to run 12 th it is cell is  (hh:mm:ss.ms) 0:00:04.912964
Time taken to run 13 th it is cell is  (hh:mm:ss.ms) 0:00:04.895371
Time taken to run 14 th it is cell is  (hh:mm:ss.ms) 0:00:

In [134]:
OUT_A ,OUT_B , OUT_C , OUT_D , OUT_E,OUT_F 


([0, 0, 13, 7, 0, 0],
 [0, 1, 17, 2, 0, 0],
 [0, 0, 14, 6, 0, 0],
 [0, 0, 16, 4, 0, 0],
 [0, 0, 17, 2, 0, 1],
 [0, 0, 19, 1, 0, 0])

In [136]:
0 + 1/20 + 14/20 + 4/20 + 1/20 
0 + 1 + 14 + 4 +1

20

In [137]:
20/120

0.16666666666666666